In [1]:
!pip install pyltr

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [46]:
!pip install more_itertools

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [3]:
import warnings
import pyltr
warnings.filterwarnings('ignore')

In [4]:
from sklearn.cluster import KMeans
import numpy as np
from more_itertools import sort_together


In [77]:
## Finding the key inside a dict
def find_matching_key_index(id, data):
    for i in range(0,len(data)):
        if data[i]['id'] == id:
            return i
    return -1


## partition data cluster wise
def partition_data_cluster_wise(X_d,Y_d,T_q,labels):
    data_dict = []
    for i in range(0, len(labels)):
        idx = find_matching_key_index(labels[i],data_dict )
        if idx == -1:
            data = {}
            data['id'] = labels[i]
            data['data'] = X_d[i:i+1]
            data['label'] = Y_d[i:i+1]
            data['tqid'] = T_q[i:i+1]
            data_dict.append(data)
        else :
            data = {}
            d =  data_dict[idx]['data']
            la =  data_dict[idx]['label']
            tq =  data_dict[idx]['tqid']
            data_dict[idx]['data'] = np.concatenate((d, X_d[i:i+1]), axis=0)
            data_dict[idx]['label'] = np.concatenate((la, Y_d[i:i+1]), axis=0)
            data_dict[idx]['tqid'] = np.concatenate((tq, T_q[i:i+1]), axis=0)

    return data_dict

## concatenate all the evaluation across clusters
def concatenate_cluster_eval(p_data):
    m_data = {}
    m_data['data'] = np.array([[np.array(xx)] for xx in  p_data[0]['eval']])
    m_data['label'] = p_data[0]['label']
    m_data['tqid'] = p_data[0]['tqid']
    for j in range(1, len(p_data)):
        dd = np.array([[np.array(xx)] for xx in  p_data[j]['eval']])
        m_data['data'] = np.concatenate((m_data['data'] ,dd), axis=0)
        m_data['label'] = np.concatenate((m_data['label'] , p_data[j]['label']), axis=0)
        m_data['tqid'] = np.concatenate((m_data['tqid'] , p_data[j]['tqid']), axis=0)
    return m_data

## train model using LamdaMart 
def train_model_using_LambdaMART(p_data):
    for j in range(0,len(p_data)):
        metric = pyltr.metrics.NDCG(k=5)
    
        model = pyltr.models.LambdaMART(
            metric=metric,
            n_estimators=1000,
            learning_rate=0.02,
            max_features=0.5,
            query_subsample=0.5,
            max_leaf_nodes=10,
            min_samples_leaf=64,
            verbose=1,
        )

        model.fit(p_data[j]['data'], p_data[j]['label'], p_data[j]['tqid'])
        p_data[j]['model'] = model
        p_data[j]['metric'] = metric
      
    return p_data

##evaluate partiotioned data
def evaluate_partitioned_data(p_data) :
    for j in range(0,len(p_data)):
        Epred = p_data[j]['model'].predict(p_data[j]['data'])
        evl_arr = evalute_preds(p_data[j]['label'],Epred,p_data[j]['tqid'], p_data[j]['metric'])
        p_data[j]['eval'] = evl_arr
    return p_data

"""
Evaluate prediction
"""
def evalute_preds(Y,Y_pred,Qids, met):
    ev_arr = []
    query_groups = pyltr.util.group.get_groups(Qids)
    for qd, a,b in query_groups:
        m = met.evaluate_preds(qd,Y[a:b], Y_pred[a:b] )
        #copy the same value b-a times
        ev_arr.extend([m]*(b-a))
    return  ev_arr

"""
Group data inside a cluster based on ids as ids needs to be  contiguous.
It may happen that after clustering the tids are not contigous then LambdaMart will throw error
"""
def group_tqids(p_data):
    for j in range(0, len(p_data)):
        aa = p_data[j]['data']
        bb = p_data[j]['label']
        cc = p_data[j]['tqid']
        ord_data = sort_together([cc,aa.flatten(),bb])
        p_data[j]['data'] = np.array([[np.array(d)] for d in ord_data[1]])
        p_data[j]['label'] = np.array(ord_data[2])
        p_data[j]['tqid'] = np.array(ord_data[0])
    return p_data

In [7]:
with open('./MQ2007/Fold1/train.txt') as trainfile, \
         open('./MQ2007/Fold1/vali.txt') as valifile, \
         open('./MQ2007/Fold1/test.txt') as evalfile:
    TX, Ty, Tqids, _ = pyltr.data.letor.read_dataset(trainfile)
    VX, Vy, Vqids, _ = pyltr.data.letor.read_dataset(valifile)
    EX, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)

#### Step 1, take the average of the features

In [8]:
##Step 1
## compute the average of the feature
f_avg = np.array([[np.mean(subarray)] for subarray in TX])
##cluster the feature into 5 clusters as stated in the research paper
km_f_avg = KMeans(n_clusters=5, random_state=0).fit(f_avg)
km_f_avg.labels_

array([2, 3, 0, ..., 1, 2, 4], dtype=int32)

#### Train the model on each cluster

In [9]:
## Arrange the data cluster wise
partitioned_data = partition_data_cluster_wise(f_avg,Ty,Tqids,km_f_avg.labels_)
##train model for each cluster
partitioned_data = train_model_using_LambdaMART(partitioned_data)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.1904       0.0594        1.30m                                         
    2       0.1925       0.0087        1.26m                                         
    3       0.1750       0.0048        1.23m                                         
    4       0.1842       0.0043        1.20m                                         
    5       0.1839      -0.0058        1.18m                                         
    6       0.1927       0.0002        1.17m                                         
    7       0.1818      -0.0068        1.18m                                         
    8       0.1967      -0.0027        1.19m                                         
    9       0.1922      -0.0012        1.19m                                         
   10       0.1707       0.0008        1.19m                                         
   15       0.1950      -0.0021        1.18m         

  480       0.4584       0.0000       31.56s                                         
  500       0.4741      -0.0007       30.29s                                         
  550       0.4864      -0.0004       27.20s                                         
  600       0.4835      -0.0001       24.13s                                         
  650       0.4682      -0.0002       21.06s                                         
  700       0.4900      -0.0003       18.01s                                         
  750       0.4541      -0.0004       14.96s                                         
  800       0.5009      -0.0008       11.95s                                         
  850       0.4726      -0.0002        8.95s                                         
  900       0.4793      -0.0004        5.97s                                         
  950       0.4746      -0.0005        2.98s                                         
 1000       0.4668       0.0000        0.00s          

  240       0.3775       0.0008        1.53m                                         
  260       0.3593       0.0004        1.49m                                         
  280       0.3639       0.0008        1.45m                                         
  300       0.3742      -0.0002        1.41m                                         
  320       0.3856       0.0001        1.37m                                         
  340       0.3912       0.0001        1.33m                                         
  360       0.3605       0.0003        1.29m                                         
  380       0.3683       0.0005        1.25m                                         
  400       0.3632       0.0009        1.21m                                         
  420       0.3711       0.0006        1.17m                                         
  440       0.3737      -0.0010        1.13m                                         
  460       0.3860      -0.0010        1.09m          

#### Evaluate partitioned data and create new features set based in the evaluation

In [10]:
##evaluate partiotioned data
partitioned_data = evaluate_partitioned_data(partitioned_data)
##merge partitioned data as mentioned in the paper [q11,q12, ..q1n, q21,q22, ...., q41,...]
merg_data = concatenate_cluster_eval(partitioned_data)

### Step 2 cluster data on new feature set created from evaluation and train it again

In [52]:
km_f_avg_1 = KMeans(n_clusters=5, random_state=0).fit(merg_data['data'])
km_f_avg_1.labels_

array([2, 2, 2, ..., 1, 1, 1], dtype=int32)

In [80]:
partitioned_data_1 = partition_data_cluster_wise(merg_data['data'],merg_data['label'],merg_data['tqid'],km_f_avg_1.labels_)
##group the ids as it needs to be contogous
partitioned_data_1 = group_tqids(partitioned_data_1)
##train model again on this dataset
partitioned_data_1 = train_model_using_LambdaMART(partitioned_data_1)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.3226       0.0660        1.84m                                         
    2       0.3771       0.0103        1.69m                                         
    3       0.3223      -0.0085        1.67m                                         
    4       0.3573      -0.0031        1.63m                                         
    5       0.3536      -0.0136        1.64m                                         
    6       0.3571       0.0115        1.62m                                         
    7       0.3719       0.0058        1.61m                                         
    8       0.3585      -0.0158        1.61m                                         
    9       0.3209       0.0175        1.60m                                         
   10       0.3330      -0.0247        1.60m                                         
   15       0.3587      -0.0020        1.60m         

  480       0.0000       0.0000       43.72s                                         
  500       0.0000       0.0000       42.03s                                         
  550       0.0000       0.0000       37.85s                                         
  600       0.0000       0.0000       33.61s                                         
  650       0.0000       0.0000       29.38s                                         
  700       0.0000       0.0000       25.18s                                         
  750       0.0000       0.0000       20.97s                                         
  800       0.0000       0.0000       16.76s                                         
  850       0.0000       0.0000       12.59s                                         
  900       0.0000       0.0000        8.40s                                         
  950       0.0000       0.0000        4.20s                                         
 1000       0.0000       0.0000        0.00s          

  240       0.0879      -0.0003       51.15s                                         
  260       0.0872      -0.0018       49.79s                                         
  280       0.0942       0.0001       48.41s                                         
  300       0.0976       0.0013       47.03s                                         
  320       0.0951       0.0000       45.70s                                         
  340       0.0785       0.0003       44.36s                                         
  360       0.0797       0.0002       43.02s                                         
  380       0.0752      -0.0027       41.68s                                         
  400       0.0856      -0.0014       40.34s                                         
  420       0.0732      -0.0018       39.01s                                         
  440       0.0877       0.0008       37.66s                                         
  460       0.0763      -0.0008       36.28s          

#### Evaluate partitioned data and create new feature set based in second clusters

In [82]:
##evaluate partiotioned data
partitioned_data_1 = evaluate_partitioned_data(partitioned_data_1)
##merge partitioned data as mentioned in the paper [q11,q12, ..q1n, q21,q22, ...., q41,...]
merg_data_1 = concatenate_cluster_eval(partitioned_data_1)

#### Step 3 cluster data for the last time  on new feature set created from evaluation and train it again

In [84]:
km_f_avg_2 = KMeans(n_clusters=5, random_state=0).fit(merg_data_1['data'])
partitioned_data_1 = partition_data_cluster_wise(merg_data_1['data'],merg_data_1['label'],merg_data_1['tqid'],km_f_avg_2.labels_)
##group the ids as it needs to be contogous
partitioned_data_2 = group_tqids(partitioned_data_1)
##train model again on this dataset
partitioned_data_2 = train_model_using_LambdaMART(partitioned_data_1)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.0016       0.0007        4.06m                                         
    2       0.0011      -0.0003        3.97m                                         
    3       0.0012       0.0002        3.95m                                         
    4       0.0007      -0.0003        3.90m                                         
    5       0.0011       0.0001        3.91m                                         
    6       0.0008      -0.0003        3.94m                                         
    7       0.0014       0.0002        4.01m                                         
    8       0.0010       0.0000        4.03m                                         
    9       0.0005      -0.0006        4.03m                                         
   10       0.0014       0.0004        4.00m                                         
   15       0.0005      -0.0006        3.95m         

  480       0.0786      -0.0021       22.96s                                         
  500       0.1056      -0.0002       22.07s                                         
  550       0.1137       0.0029       19.84s                                         
  600       0.0935       0.0000       17.65s                                         
  650       0.0970       0.0016       15.44s                                         
  700       0.1116       0.0000       13.23s                                         
  750       0.0960       0.0009       11.03s                                         
  800       0.0938       0.0000        8.81s                                         
  850       0.1222       0.0024        6.61s                                         
  900       0.1016      -0.0022        4.40s                                         
  950       0.0982      -0.0016        2.20s                                         
 1000       0.1160       0.0000        0.00s          

  240       0.3073       0.0000       30.54s                                         
  260       0.3018       0.0000       29.78s                                         
  280       0.3180       0.0000       28.95s                                         
  300       0.3059       0.0000       28.12s                                         
  320       0.3075       0.0000       27.32s                                         
  340       0.3197       0.0000       26.52s                                         
  360       0.2993       0.0000       25.69s                                         
  380       0.2961       0.0008       24.88s                                         
  400       0.2839       0.0000       24.09s                                         
  420       0.3103       0.0000       23.27s                                         
  440       0.3043       0.0000       22.48s                                         
  460       0.3053       0.0000       21.69s          

In [88]:
##evaluate partiotioned data
partitioned_data_2 = evaluate_partitioned_data(partitioned_data_2)